In [ ]:
!pip install yfinance
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

     |████████████████████████████████| 5.5MB 5.8MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=80af1a6c75974a4e7608828766d6063c2eb2c3051a4fdb2516ad11067b01b037
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
T3 = pd.DataFrame({"Close": data["Close"]})
T3['Year'] = T3.index.year 
T2 = pd.DataFrame({"cumpnl":['0'], "MA":['0'],  "STD":['0'], "Stock":['0'], "Year":['0']})

for z in range (2016, 2021, 1):
  T = T3.where(T3.Year == z)
  T = T.dropna()
  for a in range(1,20,1):
    for b in range(1,3,1):
      MA=a
      STD=b

      # Compute Moving averages of last 10 days and 30 days closing prices
      T['moving_average'] = T.Close.rolling(MA).mean()
      T['moving_std_dev'] = T.Close.rolling(MA).std()
      T['upper_band'] = T.moving_average + (T.moving_std_dev*STD)
      T['lower_band'] = T.moving_average - (T.moving_std_dev*STD)
      T['positions_long'] = np.nan
      T['positions_short'] = np.nan

      for x in range (len(T)):
        if T.Close[x] < T.lower_band[x]:
          T['positions_long'][x] = 1
        if T.Close[x] >= T.moving_average[x]:
          T['positions_long'][x] = 0
        if T.Close[x] > T.upper_band[x]:
          T['positions_short'][x] = -1
        if T.Close[x] <= T.moving_average[x]:
          T['positions_short'][x] = 0

      T.positions_long = T.positions_long.fillna(method='ffill')  
      T.positions_short = T.positions_short.fillna(method='ffill')  
      T['positions'] = T.positions_long + T.positions_short
      T['price_difference']= T.Close - T.Close.shift(1)
      T['pnl'] = T.positions.shift(1) * T.price_difference
      T['cumpnl'] = T.pnl.cumsum()
      T1 = T[['cumpnl']].tail(1)
      T1['MA'] = MA
      T1['STD'] = STD
      T1['Stock'] = Stock
      T1['Year'] = z
      T2 = T2.append(T1)

Pivot_Table1 = pd.pivot_table(T2, values ='cumpnl', index =['MA', 'STD'], 
                         columns =['Year'], aggfunc = np.sum) 

Pivot_Table1.to_csv("PV_T.csv", index=True, encoding='utf8')
from google.colab import files
files.download('PV_T.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>